In [ ]:
!pip install google-play-scraper pandas
!pip install konlpy
!pip install git+https://github.com/ssut/py-hanspell.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 16.5 MB/s eta 0:00:00
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-hhk0hv2n
  Running command git clone --filter=blob:none --quiet https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-hhk0hv2n
  Resolved https://github.com/ssut/py-hanspell.git to commit fdc6ca50c19f1c85971437a072d89d4e5ce024b8
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4810 sha256=4ac1466edca990f49f9bd509fc310ae75f658160cc65bc2b09e9c54075e93ae6
  Stored in directory: /tmp/pip-ephem-wheel-cache-tqtp7q4o/wheels/2e/43/cc/753c9e1d91affb9ea40e186cea5654fb9231deb454da6724e5
Successfully built py-hanspell


In [13]:
from google_play_scraper import search, app
import pandas as pd

# Search term for Kiggle games
DEVELOPER_NAME = 'KIGLE'

def fetch_kiggle_games(developer_name):
    # Search for games by developer name
    games = search(
        developer_name,
        n_hits=50,  # Number of results to fetch per request
        lang='en',
        country='us'
    )
    return games

def fetch_game_details(app_id):
    return app(
        app_id,
        lang='en',
        country='us'
    )

def scrape_kiggle_games():
    # Step 1: Fetch all games associated with Kiggle
    games_list = fetch_kiggle_games(DEVELOPER_NAME)

    # Step 2: Get detailed info for each game
    detailed_games_info = []
    for game in games_list:
        app_id = game['appId']
        details = fetch_game_details(app_id)
        detailed_games_info.append(details)

    # Step 3: Convert the result to a DataFrame for easy viewing/manipulation
    df = pd.DataFrame(detailed_games_info)

    # Step 4: Save the DataFrame to a CSV file or print it
    df.to_csv('kiggle_games_info.csv', index=False)
    return df


In [14]:
result = scrape_kiggle_games()
result

TypeError: 'NoneType' object is not subscriptable

In [ ]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords

# NLTK 불용어 다운로드
nltk.download('stopwords')

# 영어 불용어 목록 가져오기
english_stopwords = set(stopwords.words('english'))

# 텍스트 전처리 및 불용어 제거 함수
def preprocess_english_text(text):
    # Step 1: 특수 문자 제거 및 공백 정리
    text_rmv = re.sub(r'[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', ' ', text)
    text_rmv = re.sub(r'[\n\r]', ' ', text_rmv)  # 줄바꿈 제거
    text_rmv = re.sub(r'[^\w\s]', ' ', text_rmv)  # 특수 문자 제거
    text_rmv = re.sub(r'\s+', ' ', text_rmv)  # 다중 공백 제거

    # Step 2: 소문자 변환
    text_rmv = text_rmv.lower()

    # Step 3: 단어를 분리하여 불용어 제거
    tokens = text_rmv.split()
    filtered_words = [word for word in tokens if word not in english_stopwords]

    # Step 4: 불용어를 제거한 단어들로 다시 결합
    return ' '.join(filtered_words)

# DataFrame의 'description' 열에 전처리 적용 (영어 불용어 제거)
result['processed_description'] = result['description'].apply(preprocess_english_text)

# 전처리된 데이터 확인 (description과 processed_description 열만 출력)
print(result[['description', 'processed_description']].head())

# 결과를 CSV 파일로 저장
result.to_csv('processed_english_descriptions.csv', index=False)


                                         description  \
0  Welcome to Cocobi Life World! \r\nGet ready to...   
1  Baby Pororo and his friends are crying!\r\nTak...   
2  Welcome to Cocobi's fun park with exciting rid...   
3  The Cocobi Kindergarten is filled with the hap...   
4  "Play all our fun games in the Cocobi World ap...   

                               processed_description  
0  welcome cocobi life world get ready fun specia...  
1  baby pororo friends crying take care babies wa...  
2  welcome cocobi fun park exciting rides create ...  
3  cocobi kindergarten filled happy laughter chil...  
4  play fun games cocobi world app full games chi...  


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
result

,title,description,descriptionHTML,summary,installs,minInstalls,realInstalls,score,ratings,reviews,...,adSupported,containsAds,released,lastUpdatedOn,updated,version,comments,appId,url,processed_description
0,"Cocobi Life World - city, town",Welcome to Cocobi Life World! \r\nGet ready to...,Welcome to Cocobi Life World! <br>Get ready to...,Play the fun Life World game for kids with Coc...,"5,000,000+",5000000,5390682,3.653465,4294,78,...,True,True,"Oct 24, 2023","Mar 11, 2024",1710224417,1.0.24,[],com.kigle.cocobi.lifeworld,https://play.google.com/store/apps/details?id=...,welcome cocobi life world get ready fun specia...
1,My Pororo Baby - Kid Care Game,Baby Pororo and his friends are crying!\r\nTak...,Baby Pororo and his friends are crying!<br>Tak...,Enjoy the fun baby care game for kids! Care fo...,"100,000+",100000,264853,4.000000,337,4,...,True,True,"Feb 23, 2023","Sep 12, 2024",1726202974,1.0.9,[],com.kigle.project.pororo.babycare,https://play.google.com/store/apps/details?id=...,baby pororo friends crying take care babies wa...
2,Cocobi Theme Park - Kids game,Welcome to Cocobi's fun park with exciting rid...,Welcome to Cocobi&#39;s fun park with exciting...,An exciting theme park game with Cocobi! Enjoy...,"1,000,000+",1000000,3738356,3.727273,1209,11,...,True,True,"Nov 11, 2021","Oct 3, 2024",1728015633,1.0.16,[],com.kigle.cocobi.themepark,https://play.google.com/store/apps/details?id=...,welcome cocobi fun park exciting rides create ...
3,Cocobi Kindergarten -Preschool,The Cocobi Kindergarten is filled with the hap...,The Cocobi Kindergarten is filled with the hap...,Meet Mr. Wally and friends at the Cocobi Kinde...,"100,000+",100000,101404,0.000000,0,0,...,True,True,"Aug 13, 2024","Sep 30, 2024",1727685819,1.0.1,[],com.kigle.cocobi.kindergarten,https://play.google.com/store/apps/details?id=...,cocobi kindergarten filled happy laughter chil...
4,Cocobi World 1 - Kids Game,"""Play all our fun games in the Cocobi World ap...",&quot;Play all our fun games in the Cocobi Wor...,"Play Cocobi World 1, a fun game for kids with ...","1,000,000+",1000000,4359071,4.030000,1954,16,...,True,True,"Jan 17, 2023","Oct 4, 2024",1728032550,1.0.6,[],com.kigle.cocobi.world.one,https://play.google.com/store/apps/details?id=...,play fun games cocobi world app full games chi...
5,Cocobi Baby Care - Babysitter,"Meet Coco, Lobi, and friends as cute babies!\r...","Meet Coco, Lobi, and friends as cute babies!<b...",Fun baby care game for kids with dinosaur frie...,"5,000,000+",5000000,8042387,3.870000,3876,53,...,True,True,"Mar 16, 2023","Apr 21, 2024",1713753584,1.0.15,[],com.kigle.cocobi.babycare,https://play.google.com/store/apps/details?id=...,meet coco lobi friends cute babies take walk p...
6,Cocobi Goodnight - kids Habit,It's nighttime and time for bed!\r\nHelp the C...,It&#39;s nighttime and time for bed!<br>Help t...,Play the fun bedtime game for kids with Cocobi...,"1,000,000+",1000000,1939565,3.928571,666,6,...,True,True,"Feb 1, 2023","Sep 24, 2024",1727167008,1.0.9,[],com.kigle.cocobi.goodnight,https://play.google.com/store/apps/details?id=...,nighttime time bed help cocobi friends go bed ...
7,Cocobi Summer Vacation - Kids,Who doesn't love summer vacation?\r\nEnjoy the...,Who doesn&#39;t love summer vacation?<br>Enjoy...,Go on a summer vacation with the dinosaur fami...,"1,000,000+",1000000,3173521,3.478261,1079,6,...,True,True,"Jul 30, 2021","Oct 1, 2024",1727836604,1.2.16,[],com.kigle.cocobi.summervacation,https://play.google.com/store/apps/details?id=...,love summer vacation enjoy warm sun sandy beac...


In [ ]:
cocobi_game = result[result['title'].str.startswith('Cocobi')]
cocobi_game

,title,description,descriptionHTML,summary,installs,minInstalls,realInstalls,score,ratings,reviews,...,adSupported,containsAds,released,lastUpdatedOn,updated,version,comments,appId,url,processed_description
0,"Cocobi Life World - city, town",Welcome to Cocobi Life World! \r\nGet ready to...,Welcome to Cocobi Life World! <br>Get ready to...,Play the fun Life World game for kids with Coc...,"5,000,000+",5000000,5390682,3.653465,4294,78,...,True,True,"Oct 24, 2023","Mar 11, 2024",1710224417,1.0.24,[],com.kigle.cocobi.lifeworld,https://play.google.com/store/apps/details?id=...,welcome cocobi life world get ready fun specia...
2,Cocobi Theme Park - Kids game,Welcome to Cocobi's fun park with exciting rid...,Welcome to Cocobi&#39;s fun park with exciting...,An exciting theme park game with Cocobi! Enjoy...,"1,000,000+",1000000,3738356,3.727273,1209,11,...,True,True,"Nov 11, 2021","Oct 3, 2024",1728015633,1.0.16,[],com.kigle.cocobi.themepark,https://play.google.com/store/apps/details?id=...,welcome cocobi fun park exciting rides create ...
3,Cocobi Kindergarten -Preschool,The Cocobi Kindergarten is filled with the hap...,The Cocobi Kindergarten is filled with the hap...,Meet Mr. Wally and friends at the Cocobi Kinde...,"100,000+",100000,101404,0.000000,0,0,...,True,True,"Aug 13, 2024","Sep 30, 2024",1727685819,1.0.1,[],com.kigle.cocobi.kindergarten,https://play.google.com/store/apps/details?id=...,cocobi kindergarten filled happy laughter chil...
4,Cocobi World 1 - Kids Game,"""Play all our fun games in the Cocobi World ap...",&quot;Play all our fun games in the Cocobi Wor...,"Play Cocobi World 1, a fun game for kids with ...","1,000,000+",1000000,4359071,4.030000,1954,16,...,True,True,"Jan 17, 2023","Oct 4, 2024",1728032550,1.0.6,[],com.kigle.cocobi.world.one,https://play.google.com/store/apps/details?id=...,play fun games cocobi world app full games chi...
5,Cocobi Baby Care - Babysitter,"Meet Coco, Lobi, and friends as cute babies!\r...","Meet Coco, Lobi, and friends as cute babies!<b...",Fun baby care game for kids with dinosaur frie...,"5,000,000+",5000000,8042387,3.870000,3876,53,...,True,True,"Mar 16, 2023","Apr 21, 2024",1713753584,1.0.15,[],com.kigle.cocobi.babycare,https://play.google.com/store/apps/details?id=...,meet coco lobi friends cute babies take walk p...
6,Cocobi Goodnight - kids Habit,It's nighttime and time for bed!\r\nHelp the C...,It&#39;s nighttime and time for bed!<br>Help t...,Play the fun bedtime game for kids with Cocobi...,"1,000,000+",1000000,1939565,3.928571,666,6,...,True,True,"Feb 1, 2023","Sep 24, 2024",1727167008,1.0.9,[],com.kigle.cocobi.goodnight,https://play.google.com/store/apps/details?id=...,nighttime time bed help cocobi friends go bed ...
7,Cocobi Summer Vacation - Kids,Who doesn't love summer vacation?\r\nEnjoy the...,Who doesn&#39;t love summer vacation?<br>Enjoy...,Go on a summer vacation with the dinosaur fami...,"1,000,000+",1000000,3173521,3.478261,1079,6,...,True,True,"Jul 30, 2021","Oct 1, 2024",1727836604,1.2.16,[],com.kigle.cocobi.summervacation,https://play.google.com/store/apps/details?id=...,love summer vacation enjoy warm sun sandy beac...


In [ ]:
cocobi_game.to_csv('cocobi_game_en.csv', index=False)